# Gestión de contraseñas

En la siguiente actividad podrás entender mejor algunas herramientas que puedes usar para gestionar correctamente tus contraseñas.

# Gestores de contraseñas (KeepassXC keepassxc-cli)

`KeepassXC` es un gestor de contraseñas ampliamente utilizado por personas en todo el mundo, permite almacenar contraseñas de una forma segura en un archivo de base de datos de contraseñas que generalmente se encuentra cifrado y protegido por contraseña y/o un archivo de llaves.

Es posible instalar un cliente gráfico en Linux, Windows o Mac, sin embargo para este ejercicio utilizaremos su paquete de línea de comandos llamado `keepassxc-cli`

La siguiente ejecución nos mostrará las opciones del comando `keepassxc-cli`

In [ ]:
keepassxc-cli

En las nuevas versiones de `keepassxc-cli` es posible crear bases de datos, sin embargo en la que nos provee Binder no es posible. Para crear una nueva base de datos de contraseñas desde la línea de comandos necesitamos 2 componentes, una base de datos y un par de llaves. Si fueras a generar el par de llaves debes ejecutar:

* `ssh-keygen -N "PASSPHRASE" -C "MI PAR DE LLAVES" -f "MISLLAVES"`
Allí debes reemplazar `PASSPHRASE` por tus propia frase de paso, `MI PAR DE LLAVES` por el comentario que quieres que tenga tu par de llaves y `MISLLAVES` por el nombre que le quieres dar a tus llaves.

* `keepassxc-cli create "MIBASEDEDATOS.KDBX" --key-file "MISLLAVES"`
Allí debes reemplazar `MIBASEDEDATOS.KDBX` por el nombre que le darás a tu base de datos y `MISLLAVES` por la llave privada que acabas de crear.

Además solicitará una contraseña adicional de base de datos a la cual en mi caso le puse `L1nuxRul3s*`

Estos pasos te crearían una base de datos con sus llaves, para este ejercicio yo les proveeré todos los elementos necesarios, los podemos ver en el siguiente comando:

In [ ]:
ls $HOME/binder/resources/opt/U2/

Se puede ver allí entonces:

`MIBASEDEDATOS.KDBX` : Una base de datos de contraseñas
`MISLLAVES` : Una llave privada
`MISLLAVES.pub` : Una llave pública


Ya sabiendo donde está nuestra base de datos y sus llaves, vamos a ver el contenido en ella, primero iremos al directorio donde tenemos lo que necesitamos con el comando cd:

In [ ]:
cd $HOME/binder/resources/opt/U2/

Ahora listaremos el contenido de la base de datos con el siguiente comando:

In [ ]:
echo "L1nuxRul3s*" | keepassxc-cli ls MIBASEDEDATOS.KDBX -k MISLLAVES

Este comado envía la contraseña con un `echo` y posteriormente solicita la apertura del archivo de base de datos de contraseñas con la respectiva llave.

Nuestro siguiente paso será añadir un nuevo registro a la base de datos con el comando `keepassxc-cli add` y los parametros necesarios. En el siguiente ejemplo lo ejecutaremos sin parametros para ver todas las opciones que nos da el comando:

In [ ]:
keepassxc-cli add

Cómo pueden ver, la salida de este comando nos da unas opciones y su respectiva explicación, algunas de ellas son:

`--username
--url
--generate
--password-length
-k`

Utilizaremos esas opciones para añadir una entrada a la base de datos de contraseñas con el siguiente comando:

In [ ]:
echo "L1nuxRul3s*" | keepassxc-cli add MIBASEDEDATOS.KDBX --username "usuario@email.com" --url "https://www.email.com" "Email" --generate --password-length 12 -k MISLLAVES

Este comando acaba de añadir una entrada con las siguientes características:

* Base de datos: MIBASEDEDATOS.KDBX
* Usuario (a guardar en la entrada): usuario@email.com
* URL (a guardar en la entrada): https://www.email.com
* Nombre del registro: Email
* Contraseña: Autogenerada de 12 caracteres (si queremos añadirla debemos 

Para esta entrada se generó una contraseña aleatoria pero aún no podemos verla. Para ver el contenido de una entrada utilizamos `keepassxc-cli show`

In [ ]:
echo "L1nuxRul3s*" | keepassxc-cli show MIBASEDEDATOS.KDBX Email -k MISLLAVES

Ese comando nos mostrará el contenido del registro solicitado en la base de datos actual `MIBASEDEDATOS.KDBX`

# Validando la eficiencia de una contraseña

La eficiencia de una contraseña se puede medir por su entropía que hace referencia a la cantidad de aleatoriedad para hacer a la misma difícil de adivinar. Existen herramientas en línea que permiten calcular la entropía, incluso permiten analizar cuantos días se demoraría un ataque de fuerza bruta en adivinarla. Para este ejercicio usaremos tres herramientas:

* La primera será la misma `keepassxc-cli generate` para generar una contraseña aleatoria.
* La segunda será `keepassxc-cli estimate` 
* La tercera será `cracklib-check` estas dos últimas nos permitirán analizar la complejidad y eficiencia de una contraseña.

Comenzaremos analizando una contraseña insegura con `keepassxc-cli estimate` y luego con `cracklib-check`

In [ ]:
echo "12345678" | cracklib-check

In [ ]:
keepassxc-cli estimate "12345678"

Cómo podemos ver utilizar "12345678" es una pésima idea para una contraseña, para este ejercicio pueden reemplazar los números `12345678` por cualquier contraseña que quieran validar.

Ahora tratemos de generar una contraseña realmente compleja.

In [5]:
keepassxc-cli generate --length 16 

La anterior ejecución genera una contraseña que tiene una longitud de 16 caracteres, sin embargo en las nuevas versiones de `keepassxc-cli` podemos ejecutar algo como:

keepassxc-cli generate --length 16  --lower --upper --numeric --special --extended --exclude "|>)/"

Donde se podrá por ejemplo crear una contraseña que contenga:

* 16 caracteres
* Letras minúsculas
* Letras mayúsculas
* Caractéres numéricos
* Carácteres especiales
* Valores extendidos ASCII
* Exclusión de los caracteres |>)/

Validemos la contraseña generada seleccionandola y copiandola, luego peguemosla en el siguiente espacio que dice `"INTRODUCE_LA_CONTRASEÑA_A_VALIDAR"` para ejecutar de nuevo los comandos `keepassxc-cli estimate` y `cracklib-check` así veremos que opinan estos comandos sobre nuestra nueva contraseña.

In [ ]:
echo "INTRODUCE_LA_CONTRASEÑA_A_VALIDAR" | cracklib-check

In [ ]:
keepassxc-cli estimate "INTRODUCE_LA_CONTRASEÑA_A_VALIDAR"

Si todo salió bien, `cracklib-check` nos responderá con un `OK` y `keepassxc-cli estimate` nos dirá que la entropía de la contraseña es mucho mayor a la de `12345678`.

Si no cuentas con `keepassxc-cli generate`, siempre es posible utilizar diferentes comandos de Linux para crear contraseñas aleatorias, a continuación algunos ejemplos:

In [ ]:
openssl rand -hex 12

In [ ]:
date | md5sum

Otras opciones en Linux son:

* env LC_CTYPE=C tr -dc 'a-z0-9' < /dev/urandom | head -c24
* strings /dev/urandom | grep -o '[[:alnum:]]' | head -n 10 | tr -d '\n'

Y muchas similares, así tendrás múltiples formas de generar contraseñas seguras desde la terminal.